<a href="https://colab.research.google.com/github/Leopqs/Treinamento_de_IAs/blob/main/Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# 1) Ler o CSV (suporta ; e decimal ,)
path = "/content/base_artistas_knn - Sheet1.csv"

# Detecta separador por uma heurística simples
with open(path, "rb") as f:
    head = f.read(2048)
sep = ";" if head.count(b";") >= head.count(b",") else ","

# Tenta encodings comuns
df = None
for enc in ["utf-8", "latin1"]:
    try:
        df = pd.read_csv(path, sep=sep, encoding=enc, decimal=",")
        break
    except:
        pass

if df is None:
    raise ValueError("Não consegui ler o arquivo. Verifique o formato/encoding.")

df.columns = [c.strip() for c in df.columns]

# 2) Garantir colunas numéricas
for c in ["Obras em Vida", "Vendas em Vida", "% Vendido em Vida"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Se % não existir ou estiver vazia, calcula
if "% Vendido em Vida" not in df.columns or df["% Vendido em Vida"].isna().all():
    df["% Vendido em Vida"] = (df["Vendas em Vida"] / df["Obras em Vida"]) * 100

# 3) NOVA métrica de sucesso: 50% ou mais vendido em vida
df["Sucesso_50"] = (df["% Vendido em Vida"] >= 50).astype(int)

# 4) Features (X) e alvo (y)
# (Usando as colunas numéricas como entrada)
X = df[["Obras em Vida", "Vendas em Vida", "% Vendido em Vida"]]
y = df["Sucesso_50"]

# Remove linhas com valores faltando
mask = X.notna().all(axis=1) & y.notna()
X = X[mask]
y = y[mask].astype(int)

# 5) Split 80/20 (estratificado)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6) Modelo KNN (Vizinho Mais Próximo) + normalização
model = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

model.fit(X_train, y_train)

# 7) Previsão e métricas
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])

print("Acurácia:", acc)
print("\nMatriz de Confusão (linhas=real, colunas=previsto):")
print(pd.DataFrame(
    cm,
    index=["Real: Não (0)", "Real: Sim (1)"],
    columns=["Pred: Não (0)", "Pred: Sim (1)"]
))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Não", "Sim"]))

Acurácia: 1.0

Matriz de Confusão (linhas=real, colunas=previsto):
               Pred: Não (0)  Pred: Sim (1)
Real: Não (0)              2              0
Real: Sim (1)              0              4

Relatório de Classificação:
              precision    recall  f1-score   support

         Não       1.00      1.00      1.00         2
         Sim       1.00      1.00      1.00         4

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

